In [6]:
# import
import tensorflow
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import os

In [21]:
# data process
# read data
train = pd.read_csv('./Corona_NLP_train.csv', encoding='latin-1')
test = pd.read_csv('./Corona_NLP_train.csv', encoding='latin-1')
print(train.columns)

# get rid of irrelevant columns
train = train[['OriginalTweet', 'Sentiment']]
test = test[['OriginalTweet', 'Sentiment']]

# clean the original text by removing urls
def remove_url(original_tweet): 
    url_pattern  = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return url_pattern.sub(r'', original_tweet)

train.dropna()
train['OriginalTweet'] = train['OriginalTweet'].apply(remove_url)

# convert label to float
label_encoder = LabelEncoder()
train['Sentiment'] = label_encoder.fit_transform(train['Sentiment'])

# tokenize
tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
tokenizer.fit_on_texts(train['OriginalTweet'])
sequence = tokenizer.texts_to_sequences(train['OriginalTweet'])
# pad
padded = pad_sequences(sequence, maxlen=50, padding="post")
# split
x_train, x_valid, y_train, y_valid = train_test_split(padded, train['Sentiment'], test_size=0.2)

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')


In [22]:
# defining model
model = tensorflow.keras.Sequential()
# adding layers
# use common values for most of the hyperparameters, might test and change later...
model.add(Embedding(20000, 100, input_length=50))
model.add(Dropout(0.2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu')) 
model.add(MaxPool1D(pool_size=4))
model.add(tensorflow.keras.layers.LSTM(100))
Dense(1, activation='sigmoid')
# compile and train
model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])
model.fit(x_train, y_train, validation_data = (x_valid, y_valid), batch_size = 32, epochs = 10)

Epoch 1/10
1029/1029 [==============================] - 17s 15ms/step - loss: -7.4726 - accuracy: 0.1145 - val_loss: -8.8904 - val_accuracy: 0.1290
Epoch 2/10
1029/1029 [==============================] - 15s 15ms/step - loss: -8.9061 - accuracy: 0.1296 - val_loss: -8.8910 - val_accuracy: 0.1288
Epoch 3/10
1029/1029 [==============================] - 15s 15ms/step - loss: -8.9191 - accuracy: 0.1260 - val_loss: -8.8974 - val_accuracy: 0.1291
Epoch 4/10
1029/1029 [==============================] - 15s 14ms/step - loss: -8.9330 - accuracy: 0.1254 - val_loss: -8.9019 - val_accuracy: 0.1260
Epoch 5/10
1029/1029 [==============================] - 15s 14ms/step - loss: -9.7092 - accuracy: 0.1209 - val_loss: -10.5653 - val_accuracy: 0.1194
Epoch 6/10
1029/1029 [==============================] - 15s 14ms/step - loss: -10.9250 - accuracy: 0.1144 - val_loss: -12.2416 - val_accuracy: 0.1088
Epoch 7/10
1029/1029 [==============================] - 15s 15ms/step - loss: -14.8535 - accuracy: 0.1050 - v

In [24]:
# predict using the validation set
y_pred = model.predict(x_valid)
print(y_pred)
accuracy = model.evaluate(x_valid, y_valid)

258/258 [==============================] - 1s 2ms/step
[[1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 ...
 [1.         0.9999995  0.9999995  ... 0.99999994 1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]]
258/258 [==============================] - 1s 2ms/step - loss: -19.9995 - accuracy: 0.0660
